# Template Matching

## Librairies

In [1]:
from PIL import Image
import numpy as np
from math import sqrt

## Images et Matrices

In [2]:

waldo1 = Image.open('template1.png')
waldo2 = Image.open('template2.png')

park = Image.open('where_is_waldo.jpg')

waldo_mat_1 = np.asarray(waldo1)
waldo_mat_2 = np.asarray(waldo2)

park_mat = np.asarray(park)

## Template Matching (questions 1 et 2)

### Calcul de distance (différence) entre deux pixels

Utile pour ne pas calculer le ssd quand le premier pixel n'est déjà pas bon. ATTENTION : ça peut être un problème l'image et/ou le template sont trop bruités (par exmple si le premier pixel du template est un pixel de bruit...). Ici ça marche assez bien même avec le template bruité.

In [3]:
def distance(pix1, pix2):
    somme=0
    for i, el in enumerate(pix1) :
        somme += (el - pix2[i])**2
    return sqrt(somme)

### Fonction pour calculer le ssd entre un template une portion de l'image à une certaine position

In [4]:
def calculate_ssd(pos, template, img):
    x,y = pos
    
    mat_ext = img[y:y+template.shape[0], x:x+template.shape[1]]
    mat_dist = (mat_ext-template)**2
    val_ssd = np.sum(mat_dist)
    return val_ssd



### Fonction pour afficher le SSD sur toute l'image

NB : Cette fonction n'est pas utile mais c'est interessant de pouvoir voir le ssd sur toute l'image.

In [5]:
def ssd_maker(template_mat, img_mat):
    ssd_mat = np.zeros((img_mat.shape[0], img_mat.shape[1]))
    c=0
    ssdmin = 1000000
    for y,ligne in enumerate(img_mat) : 
        if y < 940 : 
            for x,pix in enumerate(ligne) : 
                if x<1550 :
                    #c+=1
                    ssd_mat[y][x] = calculate_ssd((x,y), template_mat, img_mat)
                    ssdmin = min(ssdmin, calculate_ssd((x,y), template_mat, img_mat))

    #ssd_gray = np.dot(ssd_mat[...,:3], [0.299, 0.587, 0.114])
    m = np.max(ssd_mat)
    ssd_mat = ssd_mat/m*255
    ima = Image.fromarray(ssd_mat)
    ima.show()


In [7]:
ssd_maker(waldo_mat_1, park_mat)

### Fonction pour chercher un template dans une image

In [21]:

def find_template(template, img_mat, threshold) :
    ssdmin = np.inf
    lim_x = img_mat.shape[0] - template.shape[0]
    lim_y = img_mat.shape[1] - template.shape[1]

    for y, ligne_img in enumerate(img_mat) : 
        if y < lim_x :
            for x, pix_img in enumerate(ligne_img) :
                if x < lim_y :
                    threshold = threshold
                    if distance(pix_img, template[0][0]) <= threshold:
                        ssd_calculated = calculate_ssd((x,y), template, img_mat)

                        if ssdmin > ssd_calculated : 
                            ssdmin = ssd_calculated
                            coo = (x,y)
    return coo
            

In [27]:
#Waldo
coo1 = find_template(waldo_mat_1, park_mat, threshold= 70)
print(coo1)

#Waldo bruité ==> besoin d'un threshold plus haut
coo2 = find_template(waldo_mat_2, park_mat, threshold= 300)
print(coo2)



/var/folders/y9/d18jjnw93dg7qrhmzqbxfgd00000gn/T/ipykernel_34526/3579178238.py:4: RuntimeWarning: overflow encountered in scalar subtract
  somme += (el - pix2[i])**2


(1258, 75)
(1258, 74)


### Fonction pour tracer un cercle autour du pattern

In [28]:
from PIL import Image, ImageDraw

def dessiner_cercle_autour_point(image, centre, template_mat):
    image_rgba = image.convert("RGBA")
    
    draw = ImageDraw.Draw(image_rgba)
    
    x, y = centre
    x += template_mat.shape[0]/2
    y += template_mat.shape[1]/2

    rayon = 70
    
    x1, y1 = x - rayon + template_mat.shape[1]/2, y - rayon + template_mat.shape[0]/2
    x2, y2 = x + rayon + template_mat.shape[1]/2, y + rayon + template_mat.shape[0]/2
    
    draw.ellipse((x1, y1, x2, y2), outline="black", width=20)
    draw.ellipse((x1, y1, x2, y2), outline="white", width=13)
    draw.ellipse((x1, y1, x2, y2), outline="red", width=8)
    draw.ellipse((x1, y1, x2, y2), outline="blue", width=4)



    
    image_rgba.show()

In [29]:
#dessiner_cercle_autour_point(park, coo1, waldo_mat_1)
dessiner_cercle_autour_point(park, coo2, waldo_mat_2)